# Cross Currency Swaps 

This notebook demonstrates how to access and use the **IR Swaps** functionalities within the **LSEG Financial Analytics SDK**.

**You will be able to:**
- Define an OTC Cross Currency IR Swap
- Evaluate the swap
- Difference between deal currency and report currency
- Display cashflows and sensivities

## Imports
Import the following necessary modules:
This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments import ir_swaps as irs
from lseg_analytics.pricing.common import AdjustableDate, RelativeAdjustableDate, ReferenceDate, FrequencyEnum, DateMovingConvention
import lseg_analytics.pricing.reference_data.floating_rate_indices as fri
import pandas as pd
from IPython.display import display

#### Define the first leg, Fixed in CNY

In [2]:
ccs_first_leg = irs.InterestRateLegDefinition(
    rate = irs.FixedRateDefinition(
        rate = irs.Rate(
            value = 3,
            unit = irs.UnitEnum.PERCENTAGE
        )
    ),
    interest_periods = irs.ScheduleDefinition(
        start_date = AdjustableDate(  # Adjustable Date could be modified depending on the conventions
            date = "2025-01-01"
        ),
        end_date = RelativeAdjustableDate( # Relative Adjustable Date could be modified depending on the conventions and tenor
            tenor = "1Y", 
            reference_date = ReferenceDate.START_DATE
        ),
        frequency = FrequencyEnum.QUARTERLY
    ),
    payment_offset = irs.OffsetDefinition( # Defines difference between the actual payment date and the interest period reference date
        tenor = "2D",
        direction = irs.DirectionEnum.FORWARD,
        reference_date = irs.CouponReferenceDateEnum.PERIOD_END_DATE,
        business_day_adjustment = irs.BusinessDayAdjustmentDefinition(calendars = ["USA", "CHN"], convention = DateMovingConvention.MODIFIED_FOLLOWING)
        
    ),
    principal = irs.PrincipalDefinition(
        currency = "CNY",
        amount = 1000000,
        final_principal_exchange = True
    ),
    payer = irs.PartyEnum.PARTY1,
    receiver = irs.PartyEnum.PARTY2
)

#### Define the second leg, Floating in USD

Before defining the leg, the code below explains how identify an index name in order to use it as a floating rate.

In [3]:
# avaibale IR curves for USD
print(fri.search(tags=["currency:USD"]))

[{'type': 'FloatingRateIndex', 'id': '67b1858d-4113-45d7-b401-8de42bd3fec0', 'location': {'space': 'LSEG', 'name': 'USD_BMA_FIX_7D'}, 'description': {'summary': 'SIFMA Fix', 'tags': ['currency:USD', 'indexTenor:7D', 'sourceLongName:Refinitiv', 'sourceShortName:RFTB']}}, {'type': 'FloatingRateIndex', 'id': '710fc04c-e793-44f4-a0f1-b133c8a37c87', 'location': {'space': 'LSEG', 'name': 'USD_FFER_ON'}, 'description': {'summary': 'Fed Funds Effective Rate', 'tags': ['currency:USD', 'indexTenor:ON', 'sourceLongName:Refinitiv', 'sourceShortName:RFTB']}}, {'type': 'FloatingRateIndex', 'id': '10a30cf1-034d-4d3d-8f88-b808cc71e15c', 'location': {'space': 'LSEG', 'name': 'USD_LIBOR_1M_IBA'}, 'description': {'summary': 'USD LIBOR (Ice Benchmark Administration)', 'tags': ['currency:USD', 'indexTenor:1M', 'sourceLongName:Ice Benchmark Administration', 'sourceShortName:IBA']}}, {'type': 'FloatingRateIndex', 'id': 'd8e2da70-7a3e-4f16-be4d-5f5b81766d95', 'location': {'space': 'LSEG', 'name': 'USD_LIBOR_1

In [4]:
# to get the correct index name, one need to use index space then the index name as below
sofr_index = [usd_index for usd_index in fri.search(tags=["currency:USD", "indexTenor:ON"]) if "SOFR" in usd_index.location.name][0]
sofr_index_name = sofr_index.location.space + "/" + sofr_index.location.name

In [5]:
ccs_second_leg = irs.InterestRateLegDefinition(
    rate = irs.FloatingRateDefinition(
        index = sofr_index_name
    ),
    interest_periods = irs.ScheduleDefinition(
        start_date = AdjustableDate(  # Adjustable Date could be modified depending on the conventions
            date = "2025-01-01"
        ),
        end_date = RelativeAdjustableDate( # Relative Adjustable Date could be modified depending on the conventions and tenor
            tenor = "1Y", 
            reference_date = ReferenceDate.START_DATE
        ),
        frequency = FrequencyEnum.QUARTERLY
    ),
    payment_offset = irs.OffsetDefinition( # Defines difference between the actual payment date and the interest period reference date
        tenor = "2D",
        direction = irs.DirectionEnum.FORWARD,
        reference_date = irs.CouponReferenceDateEnum.PERIOD_END_DATE,
        business_day_adjustment = irs.BusinessDayAdjustmentDefinition(calendars = ["USA", "CHN"], convention = DateMovingConvention.MODIFIED_FOLLOWING)
        
    ),
    principal = irs.PrincipalDefinition(
        currency = "USD",
        final_principal_exchange = True
    ),
    payer = irs.PartyEnum.PARTY2,
    receiver = irs.PartyEnum.PARTY1
)

#### Define and create the Cross Currency IR Swap

In [6]:
# Define the swap using the legs defined before
ccs_definition = irs.IrSwapDefinition(
    first_leg = ccs_first_leg,
    second_leg = ccs_second_leg
)

# Create the instrument from the definition
ccs_instrument = irs.IrSwapDefinitionInstrument(
    definition = ccs_definition
)

# Instantiate the pricing parameters
pricing_params = irs.IrPricingParameters(
    valuation_date = "2025-07-18",
    report_currency = "USD"
)

#### Evaluate the Swap

In [7]:
#  Execute the calculation using the value() function
# The 'definitions' parameter accepts a list of instruments definitions for batch processing

# Execute the calculation using the value() function with error handling
try:
    response = irs.value(
        definitions=[ccs_instrument],
        pricing_preferences = pricing_params
        
    )
    errors = [a.error for a in response.analytics if a.error]
    if errors:
        raise Exception(errors[0].message)
    print("Cross Currency IR Swap pricing execution completed")
except Exception as e:
    print(f"Price Calculation failed: {str(e)}")
    raise

Cross Currency IR Swap pricing execution completed


#### Key Sections in the `response`

 - **definitions**: Instument definitions set above. 

 - **pricingPreferences**: Valuation date.

 - **analytics**:
   - **description**: Instrument summary.
   - **valuation**: Valuation details.
   - **risk**: Sensitivities (duration, dv01 etc.)
   - **firstLeg**: description, valuation, risk and cashflows for the first leg
   - **secondLeg**: description, valuation, risk and cashflows for the second leg

In [8]:
valuation = response.analytics[0].valuation
print("Market value : ", valuation["marketValue"]["value"])
print("Accrued value : ", valuation["accrued"]["value"])
print("Clean market value : ", valuation["cleanMarketValue"]["value"])

Market value :  -22111.348806418
Accrued value :  621.595293870393
Clean market value :  -22732.9441002884


#### Deal Currency and Report Currency
To specify the deal currency, it's a the leg level by specifiying the payer as PARTY1
To specify the report currency, it's a the leg level of pricing parameters, by default is the first leg's currency

In [9]:
print("Deal Currency : ", valuation["marketValue"]["dealCurrency"]["currency"])
print("Report Currency : ", valuation["marketValue"]["reportCurrency"]["currency"])

Deal Currency :  CNY
Report Currency :  USD


#### CNY Fixed Cashflows

In [10]:
fixed_cfs = response.analytics[0].first_leg["cashflows"]
start_dates = [cf['startDate'] for cf in fixed_cfs]
end_dates = [cf['endDate'] for cf in fixed_cfs]
fixed_rates = [cf['annualRate']['value'] for cf in fixed_cfs]
currency = [cf['amount']['currency'] for cf in fixed_cfs]
amounts = [cf['amount']['value'] for cf in fixed_cfs]
paymentTypes = [cf['paymentType'] for cf in fixed_cfs]

fixed_cfs_df = pd.DataFrame({"Start Dates": start_dates, "End Dates": end_dates, 
                                "Fixed Rates": fixed_rates, "Currency": currency, "CashFlow Amounts": amounts,
                                "Payment Type": paymentTypes})
display(fixed_cfs_df)

,Start Dates,End Dates,Fixed Rates,Currency,CashFlow Amounts,Payment Type
0,2025-01-02,2025-04-01,3.0,CNY,-7315.068493,Interest
1,2025-04-01,2025-07-01,3.0,CNY,-7479.452055,Interest
2,2025-07-01,2025-10-09,3.0,CNY,-8219.178082,Interest
3,2025-10-09,2026-01-05,3.0,CNY,-7232.876712,Interest
4,2025-01-01,2026-01-05,0.0,CNY,-1000000.000000,Principal


#### USD Floating Cashflows

In [11]:
floating_cfs = response.analytics[0].second_leg["cashflows"]
start_dates = [cf['startDate'] for cf in floating_cfs]
end_dates = [cf['endDate'] for cf in floating_cfs]
floating_rates = [cf['annualRate']['value'] for cf in floating_cfs]
currency = [cf['amount']['currency'] for cf in floating_cfs]
amounts = [cf['amount']['value'] for cf in floating_cfs]
paymentTypes = [cf['paymentType'] for cf in floating_cfs]

floating_cfs_df = pd.DataFrame({"Start Dates": start_dates, "End Dates": end_dates, 
                                "Floating Rates": floating_rates, "Currency": currency, "CashFlow Amounts": amounts,
                                "Payment Type": paymentTypes})
display(floating_cfs_df)

,Start Dates,End Dates,Floating Rates,Currency,CashFlow Amounts,Payment Type
0,2025-01-02,2025-04-01,4.351480,USD,1473.644605,Interest
1,2025-04-01,2025-07-01,4.354492,USD,1507.803163,Interest
2,2025-07-01,2025-10-09,4.344141,USD,1652.987565,Interest
3,2025-10-09,2026-01-05,4.096154,USD,1371.591048,Interest
4,2025-01-01,2026-01-05,0.000000,USD,136983.487418,Principal


#### Sensitivities of the Swap

In [12]:
risks = response.analytics[0].risk
risk_df = pd.DataFrame({"Fields": list(risks.keys()), "Values": [risks[item]["value"] for item in risks]})
display(risk_df)

,Fields,Values
0,duration,-0.239136
1,modifiedDuration,-0.458625
2,benchmarkHedgeNotional,0.000000
3,annuity,-51.202051
4,dv01,-46.202017
5,pv01,-46.202017
6,br01,0.000000


### Note:
#### In order to modify the swap into float to float basis swap, or fixed to fixed, one can use the same steps, and changes the leg types to do so.